In [6]:
# 필요한 라이브러리 임포트
import pandas as pd              # 데이터 분석 및 조작
import numpy as np               # 수치 계산
import requests                  # HTTP 요청 (웹 크롤링)
from bs4 import BeautifulSoup   # HTML 파싱
import time                     # 크롤링 지연을 위한 시간 제어
import re                       # 정규 표현식
import matplotlib.pyplot as plt  # 기본 시각화
import seaborn as sns           # 고급 시각화
from wordcloud import WordCloud # 워드클라우드 생성
import warnings
import sqlite3                  # 데이터베이스 저장

# 한글 폰트 설정 (matplotlib에서 한글 표시)
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
plt.rcParams['axes.unicode_minus'] = False    # 마이너스 기호 깨짐 방지

# 경고 메시지 숨기기
warnings.filterwarnings('ignore')

print("✅ 라이브러리 임포트 완료")


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# 1. 공공도서관 현황 샘플 데이터 생성
# 실제로는 공공데이터포털에서 CSV 파일을 다운로드하여 사용
# library_df = pd.read_csv('library_info.csv')

# 샘플 데이터 생성 (실제 분석 시에는 공공데이터 활용)
library_data = {
    '도서관명': [
        '중앙도서관', '시립도서관', '구립도서관', '강남도서관', '서초도서관',
        '강동도서관', '송파도서관', '영등포도서관', '마포도서관', '은평도서관',
        '노원도서관', '도봉도서관', '성북도서관', '종로도서관', '중구도서관',
        '용산도서관', '성동도서관', '광진도서관', '동대문도서관', '중랑도서관'
    ],
    '지역': [
        '서울시 중구', '서울시 강남구', '서울시 서초구', '서울시 강남구', '서울시 서초구',
        '서울시 강동구', '서울시 송파구', '서울시 영등포구', '서울시 마포구', '서울시 은평구',
        '서울시 노원구', '서울시 도봉구', '서울시 성북구', '서울시 종로구', '서울시 중구',
        '서울시 용산구', '서울시 성동구', '서울시 광진구', '서울시 동대문구', '서울시 중랑구'
    ],
    '건립연도': [1995, 2001, 2005, 2010, 2008, 2015, 2012, 2007, 2009, 2011,
               2014, 2016, 2006, 1998, 2003, 2013, 2017, 2018, 2004, 2019],
    '시설면적(㎡)': [3500, 2800, 2200, 4200, 3800, 3100, 2900, 2600, 3300, 2700,
                  3600, 2400, 2800, 4000, 3200, 2500, 2300, 2100, 2700, 2000],
    '운영시간': ['09:00-22:00', '09:00-20:00', '09:00-18:00', '09:00-21:00', '09:00-21:00',
              '09:00-20:00', '09:00-19:00', '09:00-18:00', '09:00-21:00', '09:00-20:00',
              '09:00-22:00', '09:00-18:00', '09:00-20:00', '09:00-21:00', '09:00-19:00',
              '09:00-20:00', '09:00-18:00', '09:00-19:00', '09:00-20:00', '09:00-18:00'],
    '장서수(권)': [120000, 85000, 65000, 150000, 135000, 95000, 88000, 70000, 105000, 78000,
                110000, 62000, 82000, 140000, 98000, 75000, 68000, 58000, 80000, 55000]
}

# 데이터프레임 생성
library_df = pd.DataFrame(library_data)

# 운영시간을 시간 단위로 계산하는 함수
def calculate_operating_hours(time_str):
    """운영시간 문자열을 받아 총 운영시간(시간)을 계산"""
    try:
        start, end = time_str.split('-')
        start_hour = int(start.split(':')[0])
        end_hour = int(end.split(':')[0])
        return end_hour - start_hour
    except:
        return 0

# 운영시간(시간) 컬럼 추가
library_df['운영시간(시간)'] = library_df['운영시간'].apply(calculate_operating_hours)

# 도서관 건립 경과년수 계산
current_year = 2025
library_df['건립경과년수'] = current_year - library_df['건립연도']

print("📚 공공도서관 데이터 생성 완료")
print(f"📊 총 도서관 수: {len(library_df)}개")
print("\n🔍 데이터 미리보기:")
print(library_df.head())


In [ ]:
# 2. 네이버 블로그 리뷰 크롤링 함수 정의
def get_blog_reviews(library_name, max_results=5):
    """
    네이버 블로그에서 특정 도서관에 대한 리뷰를 크롤링하는 함수
    
    Parameters:
    - library_name (str): 도서관명
    - max_results (int): 수집할 최대 리뷰 개수
    
    Returns:
    - list: 리뷰 제목 리스트
    """
    
    try:
        # 검색 쿼리 생성 (도서관명 + "도서관" 키워드)
        query = f"{library_name} 도서관"
        url = f"https://search.naver.com/search.naver?where=blog&query={query}"
        
        # User-Agent 헤더 설정 (봇 차단 방지)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        # HTTP 요청
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # HTTP 에러 체크
        
        # HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 블로그 제목 추출 (네이버 블로그 검색 결과 구조에 따라 선택자 조정 필요)
        # 실제 선택자는 네이버 검색 결과 페이지 구조 분석 후 수정 필요
        titles = []
        
        # 여러 가능한 선택자 시도
        selectors = [
            '.title_link',  # 일반적인 제목 링크
            '.blog_title', 
            '.tit',
            '.title',
            'a[href*="blog.naver.com"]'
        ]
        
        for selector in selectors:
            elements = soup.select(selector)
            if elements:
                titles = [elem.get_text().strip() for elem in elements[:max_results]]
                break
        
        # 크롤링 결과가 없을 경우 빈 리스트 반환
        if not titles:
            print(f"⚠️  {library_name}: 리뷰 데이터를 찾을 수 없습니다.")
            return []
            
        print(f"✅ {library_name}: {len(titles)}개 리뷰 수집 완료")
        return titles[:max_results]
        
    except requests.RequestException as e:
        print(f"❌ {library_name}: 네트워크 오류 - {str(e)}")
        return []
    except Exception as e:
        print(f"❌ {library_name}: 크롤링 오류 - {str(e)}")
        return []

# 크롤링 안전을 위한 지연 함수
def safe_crawling_delay(min_delay=1, max_delay=3):
    """크롤링 간 랜덤 지연시간 설정"""
    delay = np.random.uniform(min_delay, max_delay)
    time.sleep(delay)
    
print("🔧 블로그 크롤링 함수 정의 완료")


In [ ]:
# 3. 블로그 리뷰 크롤링 실행 (샘플 데이터 사용)
# 실제 크롤링은 네트워크 상황과 네이버 정책에 따라 결과가 달라질 수 있음
# 여기서는 분석을 위한 샘플 리뷰 데이터를 생성

print("🔍 블로그 리뷰 크롤링 시작...")

# 실제 크롤링을 시도하되, 실패할 경우 샘플 데이터 사용
review_data = {}

# 샘플 리뷰 데이터 (실제 크롤링 실패 시 사용)
sample_reviews = {
    '중앙도서관': ['넓고 쾌적한 도서관', '열람실이 조용해서 좋아요', '책이 정말 많네요', '시설이 깔끔합니다', '직원분들이 친절해요'],
    '시립도서관': ['접근성이 좋습니다', '주차장이 편리해요', '아이들과 함께 가기 좋아요', '프로그램이 다양해요', '신간도서가 빨리 들어와요'],
    '구립도서관': ['동네에 있어서 편해요', '소규모지만 아늑합니다', '조용하고 분위기 좋아요', '학습환경이 좋습니다', '시설이 현대적이에요'],
    '강남도서관': ['시설이 최신식이에요', '좌석이 넓고 편안해요', '디지털 서비스가 좋아요', '카페도 있어서 편리해요', '교통이 편리합니다'],
    '서초도서관': ['깨끗하고 밝아요', '열람실 환경이 최고', '직원 서비스가 우수해요', '온라인 예약 시스템이 편해요', '주변 환경이 좋아요']
}

# 모든 도서관에 대해 리뷰 수집 시도
for library_name in library_df['도서관명']:
    try:
        # 실제 크롤링 시도 (주석 해제 시 실행)
        # reviews = get_blog_reviews(library_name, max_results=5)
        # safe_crawling_delay()  # 크롤링 간 지연
        
        # 샘플 데이터 사용 (실제 크롤링 대신)
        if library_name in sample_reviews:
            reviews = sample_reviews[library_name]
        else:
            # 기타 도서관에 대한 일반적인 샘플 리뷰
            reviews = [
                f'{library_name} 이용 후기',
                '조용하고 좋은 환경',
                '시설이 깔끔해요',
                '직원분들이 친절합니다',
                '책이 다양해서 좋아요'
            ]
        
        review_data[library_name] = reviews
        print(f"✅ {library_name}: {len(reviews)}개 리뷰 수집")
        
    except Exception as e:
        print(f"❌ {library_name}: 리뷰 수집 실패 - {str(e)}")
        review_data[library_name] = []

print(f"\n📊 리뷰 크롤링 완료 - 총 {len(review_data)}개 도서관 리뷰 수집")


In [ ]:
# 4. 리뷰 데이터 전처리 및 도서관 데이터와 병합

# 리뷰 개수 계산
library_df['리뷰개수'] = library_df['도서관명'].map(lambda x: len(review_data.get(x, [])))

# 리뷰 텍스트 합치기 (전체 리뷰를 하나의 문자열로)
library_df['전체리뷰'] = library_df['도서관명'].map(
    lambda x: ' '.join(review_data.get(x, []))
)

# 리뷰에서 긍정 키워드 개수 세기
positive_keywords = ['좋', '편리', '깨끗', '친절', '쾌적', '넓', '조용', '최고', '우수', '다양', '편안', '현대적', '최신']

def count_positive_keywords(text):
    """텍스트에서 긍정 키워드 개수를 세는 함수"""
    count = 0
    for keyword in positive_keywords:
        count += text.count(keyword)
    return count

library_df['긍정키워드수'] = library_df['전체리뷰'].apply(count_positive_keywords)

# 도서관별 만족도 점수 계산 (긍정 키워드 비율 기반)
library_df['만족도점수'] = library_df['긍정키워드수'] / (library_df['리뷰개수'] + 1) * 10  # +1은 0으로 나누기 방지

# 시설 규모 등급 분류
def classify_library_size(area):
    """시설면적에 따른 도서관 규모 분류"""
    if area >= 4000:
        return '대형'
    elif area >= 3000:
        return '중형'
    else:
        return '소형'

library_df['규모등급'] = library_df['시설면적(㎡)'].apply(classify_library_size)

# 운영시간 등급 분류
def classify_operating_hours(hours):
    """운영시간에 따른 분류"""
    if hours >= 12:
        return '장시간'
    elif hours >= 10:
        return '표준'
    else:
        return '단시간'

library_df['운영등급'] = library_df['운영시간(시간)'].apply(classify_operating_hours)

print("🔄 데이터 병합 및 전처리 완료")
print(f"📈 분석 대상 도서관: {len(library_df)}개")
print(f"📊 평균 만족도 점수: {library_df['만족도점수'].mean():.2f}")
print("\n🔍 처리된 데이터 미리보기:")
print(library_df[['도서관명', '지역', '리뷰개수', '긍정키워드수', '만족도점수', '규모등급']].head())


In [ ]:
# 5. 기본 통계 분석

print("=" * 60)
print("📊 공공도서관 현황 및 리뷰 분석 결과")
print("=" * 60)

# 기본 통계 정보
print("\n🏛️ 도서관 현황 기본 통계:")
print(f"   • 전체 도서관 수: {len(library_df)}개")
print(f"   • 평균 시설면적: {library_df['시설면적(㎡)'].mean():.0f}㎡")
print(f"   • 평균 운영시간: {library_df['운영시간(시간)'].mean():.1f}시간")
print(f"   • 평균 장서수: {library_df['장서수(권)'].mean():,.0f}권")
print(f"   • 평균 건립경과년수: {library_df['건립경과년수'].mean():.1f}년")

print("\n📝 블로그 리뷰 분석:")
print(f"   • 총 수집 리뷰 수: {library_df['리뷰개수'].sum()}개")
print(f"   • 도서관당 평균 리뷰 수: {library_df['리뷰개수'].mean():.1f}개")
print(f"   • 평균 만족도 점수: {library_df['만족도점수'].mean():.2f}/10.0")
print(f"   • 긍정 키워드 평균 개수: {library_df['긍정키워드수'].mean():.1f}개")

# 규모별 도서관 분포
print("\n🏢 도서관 규모별 분포:")
size_dist = library_df['규모등급'].value_counts()
for size, count in size_dist.items():
    print(f"   • {size}: {count}개 ({count/len(library_df)*100:.1f}%)")

# 운영시간별 분포
print("\n⏰ 운영시간별 분포:")
hour_dist = library_df['운영등급'].value_counts()
for hour_type, count in hour_dist.items():
    print(f"   • {hour_type}: {count}개 ({count/len(library_df)*100:.1f}%)")

# 만족도 상위 5개 도서관
print("\n🏆 만족도 상위 5개 도서관:")
top_libraries = library_df.nlargest(5, '만족도점수')[['도서관명', '지역', '만족도점수', '긍정키워드수']]
for idx, row in top_libraries.iterrows():
    print(f"   {row.name+1}. {row['도서관명']} ({row['지역']}) - {row['만족도점수']:.2f}점")


In [ ]:
# 6. 시각화 1: 만족도 점수와 시설 특성 관계 분석

# 그래프 스타일 설정
plt.style.use('default')
sns.set_palette("husl")

# 전체 그래프 크기 설정
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('📊 도서관 만족도와 시설 특성 관계 분석', fontsize=16, fontweight='bold')

# 1) 시설면적 vs 만족도 점수 산점도
axes[0, 0].scatter(library_df['시설면적(㎡)'], library_df['만족도점수'], 
                   alpha=0.7, s=100, c=library_df['긍정키워드수'], cmap='viridis')
axes[0, 0].set_xlabel('시설면적(㎡)')
axes[0, 0].set_ylabel('만족도 점수')
axes[0, 0].set_title('시설면적 vs 만족도 점수')
axes[0, 0].grid(True, alpha=0.3)

# 상관계수 표시
corr = library_df['시설면적(㎡)'].corr(library_df['만족도점수'])
axes[0, 0].text(0.05, 0.95, f'상관계수: {corr:.3f}', transform=axes[0, 0].transAxes, 
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 2) 운영시간 vs 만족도 점수
axes[0, 1].scatter(library_df['운영시간(시간)'], library_df['만족도점수'], 
                   alpha=0.7, s=100, c='orange')
axes[0, 1].set_xlabel('운영시간(시간)')
axes[0, 1].set_ylabel('만족도 점수')
axes[0, 1].set_title('운영시간 vs 만족도 점수')
axes[0, 1].grid(True, alpha=0.3)

# 상관계수 표시
corr2 = library_df['운영시간(시간)'].corr(library_df['만족도점수'])
axes[0, 1].text(0.05, 0.95, f'상관계수: {corr2:.3f}', transform=axes[0, 1].transAxes,
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 3) 규모별 만족도 분포 박스플롯
sns.boxplot(data=library_df, x='규모등급', y='만족도점수', ax=axes[1, 0])
axes[1, 0].set_title('도서관 규모별 만족도 분포')
axes[1, 0].set_xlabel('도서관 규모')
axes[1, 0].set_ylabel('만족도 점수')

# 4) 건립연도 vs 만족도 점수
axes[1, 1].scatter(library_df['건립연도'], library_df['만족도점수'], 
                   alpha=0.7, s=100, c='red')
axes[1, 1].set_xlabel('건립연도')
axes[1, 1].set_ylabel('만족도 점수')
axes[1, 1].set_title('건립연도 vs 만족도 점수')
axes[1, 1].grid(True, alpha=0.3)

# 상관계수 표시
corr3 = library_df['건립연도'].corr(library_df['만족도점수'])
axes[1, 1].text(0.05, 0.95, f'상관계수: {corr3:.3f}', transform=axes[1, 1].transAxes,
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("📈 시각화 1 완료: 만족도 점수와 시설 특성 관계 분석")


In [ ]:
# 7. 시각화 2: 리뷰 키워드 분석 (워드클라우드)

# 모든 리뷰 텍스트 합치기
all_reviews_text = ' '.join(library_df['전체리뷰'].tolist())

# 한글 워드클라우드를 위한 폰트 설정 (시스템에 따라 조정 필요)
try:
    # Windows 기본 한글 폰트
    font_path = 'C:/Windows/Fonts/malgun.ttf'
    
    # 워드클라우드 생성
    wordcloud = WordCloud(
        font_path=font_path,
        width=800, 
        height=400,
        background_color='white',
        max_words=100,
        colormap='viridis'
    ).generate(all_reviews_text)
    
    # 시각화
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('📝 도서관 리뷰 키워드 워드클라우드', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    print("☁️ 워드클라우드 생성 완료")
    
except Exception as e:
    print(f"⚠️ 워드클라우드 생성 실패: {str(e)}")
    print("💡 한글 폰트 경로를 확인하거나 wordcloud 라이브러리를 설치해주세요.")
    
    # 대안: 키워드 빈도 막대그래프
    print("🔄 대안으로 키워드 빈도 분석을 진행합니다...")
    
    # 긍정 키워드별 빈도 계산
    keyword_counts = {}
    for keyword in positive_keywords:
        count = all_reviews_text.count(keyword)
        if count > 0:
            keyword_counts[keyword] = count
    
    # 빈도 높은 상위 10개 키워드 시각화
    if keyword_counts:
        sorted_keywords = dict(sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True)[:10])
        
        plt.figure(figsize=(10, 6))
        bars = plt.bar(sorted_keywords.keys(), sorted_keywords.values(), color='skyblue', edgecolor='navy')
        plt.title('🔤 도서관 리뷰 주요 긍정 키워드 빈도', fontsize=14, fontweight='bold')
        plt.xlabel('키워드')
        plt.ylabel('빈도')
        plt.xticks(rotation=45)
        
        # 막대 위에 값 표시
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    f'{int(height)}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
        
        print("📊 키워드 빈도 분석 완료")


In [ ]:
# 8. 시각화 3: 지역별 및 특성별 분석

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🗺️ 지역별 및 특성별 도서관 분석', fontsize=16, fontweight='bold')

# 1) 지역별 도서관 수
district_counts = library_df['지역'].str.split().str[1].value_counts()  # '서울시 강남구'에서 '강남구' 추출
axes[0, 0].bar(district_counts.index[:10], district_counts.values[:10], color='lightcoral')
axes[0, 0].set_title('지역별 도서관 수 (상위 10개 구)')
axes[0, 0].set_xlabel('지역')
axes[0, 0].set_ylabel('도서관 수')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2) 지역별 평균 만족도
district_satisfaction = library_df.groupby(library_df['지역'].str.split().str[1])['만족도점수'].mean().sort_values(ascending=False)
axes[0, 1].bar(district_satisfaction.index[:10], district_satisfaction.values[:10], color='lightgreen')
axes[0, 1].set_title('지역별 평균 만족도 (상위 10개 구)')
axes[0, 1].set_xlabel('지역')
axes[0, 1].set_ylabel('평균 만족도 점수')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3) 운영등급별 만족도 비교
operation_satisfaction = library_df.groupby('운영등급')['만족도점수'].mean()
colors = ['#FF9999', '#66B2FF', '#99FF99']
axes[1, 0].bar(operation_satisfaction.index, operation_satisfaction.values, color=colors)
axes[1, 0].set_title('운영시간별 평균 만족도')
axes[1, 0].set_xlabel('운영등급')
axes[1, 0].set_ylabel('평균 만족도 점수')

# 값 표시
for i, v in enumerate(operation_satisfaction.values):
    axes[1, 0].text(i, v + 0.1, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')

# 4) 장서수 vs 만족도 산점도 (규모별 색상 구분)
size_colors = {'대형': 'red', '중형': 'blue', '소형': 'green'}
for size in library_df['규모등급'].unique():
    data = library_df[library_df['규모등급'] == size]
    axes[1, 1].scatter(data['장서수(권)'], data['만족도점수'], 
                       label=size, alpha=0.7, s=100, c=size_colors[size])

axes[1, 1].set_xlabel('장서수(권)')
axes[1, 1].set_ylabel('만족도 점수')
axes[1, 1].set_title('장서수 vs 만족도 (규모별)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# 상관계수 표시
corr4 = library_df['장서수(권)'].corr(library_df['만족도점수'])
axes[1, 1].text(0.05, 0.95, f'상관계수: {corr4:.3f}', transform=axes[1, 1].transAxes,
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("🗺️ 시각화 3 완료: 지역별 및 특성별 분석")


In [ ]:
# 9. 데이터 저장 (SQLite 데이터베이스 + CSV 파일)

# SQLite 데이터베이스 저장
try:
    conn = sqlite3.connect('library_analysis.db')
    
    # 메인 분석 결과 테이블 저장
    library_df.to_sql('library_review_analysis', conn, if_exists='replace', index=False)
    
    # 요약 통계 테이블 생성 및 저장
    summary_stats = {
        '항목': ['총 도서관 수', '평균 시설면적(㎡)', '평균 운영시간(시간)', 
               '평균 장서수(권)', '평균 만족도점수', '총 리뷰 수'],
        '값': [len(library_df), 
              library_df['시설면적(㎡)'].mean(),
              library_df['운영시간(시간)'].mean(),
              library_df['장서수(권)'].mean(),
              library_df['만족도점수'].mean(),
              library_df['리뷰개수'].sum()]
    }
    
    summary_df = pd.DataFrame(summary_stats)
    summary_df.to_sql('analysis_summary', conn, if_exists='replace', index=False)
    
    conn.close()
    print("✅ SQLite 데이터베이스 저장 완료: library_analysis.db")
    
except Exception as e:
    print(f"❌ 데이터베이스 저장 실패: {str(e)}")

# CSV 파일 저장
try:
    # 전체 분석 결과 저장
    library_df.to_csv('library_review_analysis.csv', encoding='utf-8-sig', index=False)
    print("✅ CSV 파일 저장 완료: library_review_analysis.csv")
    
    # 만족도 상위 도서관 리스트 저장
    top_libraries = library_df.nlargest(10, '만족도점수')[
        ['도서관명', '지역', '시설면적(㎡)', '운영시간(시간)', '장서수(권)', 
         '만족도점수', '긍정키워드수', '규모등급']
    ]
    top_libraries.to_csv('top_satisfaction_libraries.csv', encoding='utf-8-sig', index=False)
    print("✅ 상위 만족도 도서관 CSV 저장 완료: top_satisfaction_libraries.csv")
    
except Exception as e:
    print(f"❌ CSV 파일 저장 실패: {str(e)}")

print("\n💾 모든 데이터 저장 작업 완료")


In [ ]:
# 10. 주요 인사이트 도출 및 정책 제안

print("=" * 80)
print("🔍 공공도서관 현황 및 블로그 리뷰 분석 - 주요 인사이트")
print("=" * 80)

# 주요 인사이트 도출
insights = []

# 1. 시설 특성과 만족도 관계 분석
area_corr = library_df['시설면적(㎡)'].corr(library_df['만족도점수'])
hours_corr = library_df['운영시간(시간)'].corr(library_df['만족도점수'])
year_corr = library_df['건립연도'].corr(library_df['만족도점수'])

print("\n📊 1. 시설 특성과 만족도 관계 분석")
print(f"   • 시설면적과 만족도 상관계수: {area_corr:.3f}")
print(f"   • 운영시간과 만족도 상관계수: {hours_corr:.3f}")
print(f"   • 건립연도와 만족도 상관계수: {year_corr:.3f}")

if area_corr > 0.3:
    insights.append("시설면적이 클수록 이용자 만족도가 높은 경향")
if hours_corr > 0.3:
    insights.append("운영시간이 길수록 이용자 만족도가 높은 경향")
if year_corr > 0.3:
    insights.append("최근 건립된 도서관일수록 만족도가 높은 경향")

# 2. 규모별 만족도 차이
size_satisfaction = library_df.groupby('규모등급')['만족도점수'].mean()
print(f"\n🏢 2. 도서관 규모별 평균 만족도")
for size, score in size_satisfaction.items():
    print(f"   • {size}: {score:.2f}점")

best_size = size_satisfaction.idxmax()
insights.append(f"도서관 규모 중 {best_size} 도서관의 만족도가 가장 높음")

# 3. 운영시간과 만족도 관계
hours_satisfaction = library_df.groupby('운영등급')['만족도점수'].mean()
print(f"\n⏰ 3. 운영시간별 평균 만족도")
for hours_type, score in hours_satisfaction.items():
    print(f"   • {hours_type}: {score:.2f}점")

best_hours = hours_satisfaction.idxmax()
insights.append(f"운영시간 중 {best_hours} 운영 도서관의 만족도가 가장 높음")

# 4. 지역별 만족도 차이
district_satisfaction = library_df.groupby(library_df['지역'].str.split().str[1])['만족도점수'].mean()
top_district = district_satisfaction.idxmax()
bottom_district = district_satisfaction.idxmin()

print(f"\n🗺️ 4. 지역별 만족도 분석")
print(f"   • 최고 만족도 지역: {top_district} ({district_satisfaction[top_district]:.2f}점)")
print(f"   • 최저 만족도 지역: {bottom_district} ({district_satisfaction[bottom_district]:.2f}점)")

insights.append(f"{top_district}는 도서관 서비스 만족도가 가장 높은 지역")
insights.append(f"{bottom_district}는 도서관 서비스 개선이 필요한 지역")

# 5. 리뷰 키워드 분석
keyword_counts = {}
for keyword in positive_keywords:
    count = all_reviews_text.count(keyword)
    if count > 0:
        keyword_counts[keyword] = count

top_keywords = dict(sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True)[:5])
print(f"\n💬 5. 주요 긍정 키워드 (상위 5개)")
for keyword, count in top_keywords.items():
    print(f"   • '{keyword}': {count}회 언급")

most_mentioned = max(keyword_counts, key=keyword_counts.get)
insights.append(f"이용자들이 가장 자주 언급하는 긍정 요소: '{most_mentioned}'")

print(f"\n✨ 도출된 주요 인사이트:")
for i, insight in enumerate(insights, 1):
    print(f"   {i}. {insight}")

print("\n" + "=" * 80)
print("🏛️ 정책 제안 및 개선 방안")
print("=" * 80)

# 정책 제안
recommendations = [
    {
        "분야": "시설 인프라",
        "제안": [
            "소형 도서관의 시설 확장 및 리모델링 지원",
            "신규 도서관 건립 시 충분한 면적 확보",
            "기존 노후 도서관의 현대적 시설 업그레이드"
        ]
    },
    {
        "분야": "운영 서비스",
        "제안": [
            "도서관 운영시간 연장 (특히 단시간 운영 도서관)",
            "주말 및 공휴일 운영시간 확대",
            "야간 이용자를 위한 24시간 열람실 운영"
        ]
    },
    {
        "분야": "지역별 균형 발전",
        "제안": [
            f"{bottom_district} 지역 도서관 서비스 질 개선",
            "만족도 낮은 지역에 신규 도서관 건립 우선 고려",
            f"{top_district} 지역의 우수 사례 벤치마킹 및 확산"
        ]
    },
    {
        "분야": "이용자 만족도 향상",
        "제안": [
            f"'{most_mentioned}' 요소 강화를 통한 만족도 제고",
            "정기적인 이용자 피드백 수집 및 반영 시스템 구축",
            "도서관별 특성화 프로그램 개발 및 운영"
        ]
    }
]

for rec in recommendations:
    print(f"\n📋 {rec['분야']} 개선 방안:")
    for i, suggestion in enumerate(rec['제안'], 1):
        print(f"   {i}. {suggestion}")

print(f"\n🎯 우선 개선 대상 도서관:")
# 만족도 하위 5개 도서관
bottom_libraries = library_df.nsmallest(5, '만족도점수')[['도서관명', '지역', '만족도점수', '규모등급']]
for idx, row in bottom_libraries.iterrows():
    print(f"   • {row['도서관명']} ({row['지역']}) - {row['만족도점수']:.2f}점 [{row['규모등급']}]")

print("\n" + "=" * 80)
print("✅ 분석 완료 - 상세한 분석 보고서 작성을 위한 기초 자료 준비 완료")
print("=" * 80)
